## Model Selection 소개

### 학습/테스트 데이터 셋 분리 – train_test_split()

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [24]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label) # 전체 데이터를 모두 학습에 사용함
pred = dt_clf.predict(train_data) # 학습에 사용한 데이터로 예측을 수행함
print("예측 정확도:", accuracy_score(train_label, pred)) # 학습에 사용한 데이터만 썼으므로 정확도가 100%가 나옴

# 랜덤한 값을 train_data, train_label에 아래와 같이 추가하고 다시 해보면 정확도가 100보다 떨어짐
train_data = np.append(train_data, np.array([[1, 2, 3, 4]]), axis=0)
train_label = np.append(train_label, np.array([1]), axis=0)
dt_clf.fit(train_data, train_label)
pred = dt_clf.predict(train_data) # 학습에 사용한 데이터로 예측을 수행함
print("예측 정확도:", accuracy_score(train_label, pred)) # 학습에 사용한 데이터만 썼으므로 정확도가 100%가 나옴

예측 정확도: 1.0
예측 정확도: 1.0


In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [28]:
dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)

**train_test_split()**
- 전체 데이터셋을 학습 데이터셋, 테스트용 데이터셋으로 분리하기 위해 사용하는 메서드
<br />
- feature 데이터, target 데이터를 인자로 전달한다. 그 외 옵션으로는,
- test_size: 전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링할 것인지 결정한다 (default=0.25 즉, 25%)
- random_state: random_state는 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값. train_test_split()은 호출 시 무작위로 데이터를 분리하기 때문에 random_state를 지정하지 않으면 수행할 때마다 다른 학습/테스트용 데이터가 생성됨 (수업을 위해 사용하는 값인듯)

- 추가적으로, X_train, X_test처럼 피처 데이터는 대문자 X를 사용하고, y_train, y_test 처럼 타겟 데이터는 소문자 y를 사용하는 것이 관습이다

In [36]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print(f"예측 정확도: {accuracy_score(y_test, pred):.4f}")

예측 정확도: 0.9556


**판다스 DataFrame/Series도 train_test_split( )으로 분할이 가능할까?**
- 가능하다
- 추가적으로 DataFrame, Series도 fit, predict의 인풋값으로 들어갈 수 있다

In [38]:
import pandas as pd

# 이렇게 하면 sklearn 내장 데이터를 DataFrame으로 바꿀 수 있다
# 피처 데이터로 데이터프레임을 만들어 주고 컬럼명은 feature_names에서 가져온다
iris_df = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
# 만들어진 데이터프레임에 target컬럼을 만들고 타겟 값을 넣어준다 (컬럼 추가)
iris_df["target"] = iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [48]:
# 데이터 프레임의 모든 행과 마지막 1개 컬럼 제외한 모든 컬럼 선택 => 데이터 프레임 반환
ftr_df = iris_df.iloc[:, :-1]

# 데이터 프레임의 모든 행과 마지막 1개 컬럼만 선택 => 시리즈 반환
tgt_df = iris_df.iloc[:, -1]

print(type(ftr_df))
print(type(tgt_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [49]:
X_train, X_test, y_train, y_test = train_test_split(ftr_df, tgt_df, test_size=0.3, random_state=121)

In [51]:
# 학습과 예측에서도 DataFrame 형태로 값을 전달해도 괜찮은 경우들이 있다
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print(f"예측 정확도: {accuracy_score(y_test, pred):.4f}")

예측 정확도: 0.9556


### 교차 검증 (Cross Validation)

모델을 학습시키고 성능을 측정하기 위해 모든 데이터를 학습 데이터 세트와 테스트 데이터 세트로 나눈다. 이때, 한번 나눠진 학습 데이터 세트를 학습 데이터 세트와 검증 데이터 세트로 다시 분할해서 실제 테스트를 하기 전에 학습과 검증을 미리 여러 번 해보면서 성능을 테스트해보는 방식이 교차 검증 기법이다.
- **목적**: 과적합(overfitting)을 방지하고 조금 더 generalization이 가능한 모델인지 여부를 평가하기 위함. 만약 교차 검증을 하지 않는다면 하나의 데이터 셋에 과적합되고 다른 성질의 데이터셋에 예측력을 제대로 갖추지 못할 가능성이 생기기 때문이다
- 즉, **교차 검증을 하면 모델의 accuracy는 낮아진다**. 과적합을 막고 조금 더 일반화된 모델을 만들기 위해 낯선 데이터에 계속 노출시키는 방식이기 때문이다.

**K-Fold & Stratified K-Fold 교차 검증**
- 가장 보편적인 교차 검증 기법이다
- K=5인 경우 데이터 세트를 5개로 나누고 학습 데이터 세트와 검증 세트를 바꿔가면서 총 5번 학습/검증을 반복함을 뜻한다.
- 일반 K-Fold와 Stratified K-Fold가 있는데, 일반적으로 분류에서는 Stratified K-Fold를 많이 사용한다. 왜냐하면, 불균형한 분포도를 가진 레이블 데이터 집합이 있는 경우(신용카드 사기 기록 같은 드문 데이터 세트), 분할된 세트에서 드문 레이블 값에 대해 제대로 학습을 하지 못할 수 있기 때문이다. 따라서 어느정도 학습데이터/검증데이터 세트가 가지는 레이블 분포도가 비슷해야 하는데, 이를 가능하게 해주는 것이 Stratified K-Fold 교차 검증 방식이다. 

### K-Fold

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [4]:
iris = load_iris()
features = iris.data
target = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

In [6]:
kfold = KFold(n_splits=5)
cv_accuracy = []
print("붓꽃 데이터 세트 크기:", features.shape[0])

붓꽃 데이터 세트 크기: 150


In [18]:
n_iter = 0

# k-fold에서는 features만 인자로 전달하면 된다. 
# stratified k-fold와 달리 타겟값의 고른 분포를 고려하지 않기 때문이다
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f"#{n_iter} 교차 검증 정확도: {accuracy}, 학습 데이터 크기: {train_size}, 검증 데이터 크기: {test_size}")
    print(f"#{n_iter} 검증 세트 인덱스: {test_index}")
    
    cv_accuracy.append(accuracy)
    
print(np.mean(cv_accuracy))

#1 교차 검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#2 교차 검증 정확도: 0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#3 교차 검증 정확도: 0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#4 교차 검증 정확도: 0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#5 교차 검증 정확도: 0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
0.9


### Stratified K-Fold

In [26]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["target"] = iris.target
iris_df["target"].value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [28]:
# 먼저 K=3인 K Fold 검증을 수행해본다
kfold = KFold(n_splits=3)

n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    # 타겟 값이 어떻게 쪼개지는지 보기 위해 타겟 값만 뽑아본다
    target_train = iris_df['target'].iloc[train_index]
    target_test = iris_df['target'].iloc[test_index]
    print(f"#{n_iter}번째 교차검증")
    print(f"학습 데이터 레이블 분포:", target_train.value_counts())
    print(f"검증 데이터 레이블 분포:", target_test.value_counts())
    
# 결과를 보면 레이블 1, 2값만 학습 데이터에 있는데, 검증에서는 0을 맞춰야하는 상황 등이 발생한다

#1번째 교차검증
학습 데이터 레이블 분포: target
1    50
2    50
Name: count, dtype: int64
검증 데이터 레이블 분포: target
0    50
Name: count, dtype: int64
#2번째 교차검증
학습 데이터 레이블 분포: target
0    50
2    50
Name: count, dtype: int64
검증 데이터 레이블 분포: target
1    50
Name: count, dtype: int64
#3번째 교차검증
학습 데이터 레이블 분포: target
0    50
1    50
Name: count, dtype: int64
검증 데이터 레이블 분포: target
2    50
Name: count, dtype: int64


In [32]:
# 이번에는 Stratified K Fold로 교차 검증을 수행해본다
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

# skf.split에 타겟값을 같이 전달해서 각각 서브셋에 타겟값이 고루 분포되도록 한다
for train_index, test_index in skf.split(iris_df, iris_df['target']):
    n_iter += 1
    target_train = iris_df['target'].iloc[train_index]
    target_test = iris_df['target'].iloc[test_index]
    print(f"{n_iter}번째 교차검증")
    print("학습 데이터 레이블 분포:", target_train.value_counts())
    print("테스트 데이터 레이블 분포:", target_test.value_counts())

# 결과를 보면 타겟 값이 고루 분포되어 있음을 알 수 있다
    

1번째 교차검증
학습 데이터 레이블 분포: target
2    34
0    33
1    33
Name: count, dtype: int64
테스트 데이터 레이블 분포: target
0    17
1    17
2    16
Name: count, dtype: int64
2번째 교차검증
학습 데이터 레이블 분포: target
1    34
0    33
2    33
Name: count, dtype: int64
테스트 데이터 레이블 분포: target
0    17
2    17
1    16
Name: count, dtype: int64
3번째 교차검증
학습 데이터 레이블 분포: target
0    34
1    33
2    33
Name: count, dtype: int64
테스트 데이터 레이블 분포: target
1    17
2    17
0    16
Name: count, dtype: int64


In [42]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=5)
n_iter = 0
cv_accuracy = []

iris = load_iris()
features = iris.data
target = iris.target

dt_clf = DecisionTreeClassifier(random_state=156)

for train_index, test_index in skfold.split(features, target):
    # split을 하면 인덱스를 반환해주는데 이를 이용해서 학습/검증용 데이터를 각각 추출해낸다
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # 학습 & 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # 루프별 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    
    print(f"#{n_iter}번째 교차검증 정확도: {accuracy}")
    print(f"#{n_iter}번째 검증 세트 인덱스: {test_index}")
    
    cv_accuracy.append(accuracy)

print("교차 검증별 정확도", np.round(cv_accuracy, 4))
print("평균 검증 정확도", np.round(np.mean(cv_accuracy), 4))

#1번째 교차검증 정확도: 0.9667
#1번째 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]
#2번째 교차검증 정확도: 0.9667
#2번째 검증 세트 인덱스: [ 10  11  12  13  14  15  16  17  18  19  60  61  62  63  64  65  66  67
  68  69 110 111 112 113 114 115 116 117 118 119]
#3번째 교차검증 정확도: 0.9
#3번째 검증 세트 인덱스: [ 20  21  22  23  24  25  26  27  28  29  70  71  72  73  74  75  76  77
  78  79 120 121 122 123 124 125 126 127 128 129]
#4번째 교차검증 정확도: 0.9667
#4번째 검증 세트 인덱스: [ 30  31  32  33  34  35  36  37  38  39  80  81  82  83  84  85  86  87
  88  89 130 131 132 133 134 135 136 137 138 139]
#5번째 교차검증 정확도: 1.0
#5번째 검증 세트 인덱스: [ 40  41  42  43  44  45  46  47  48  49  90  91  92  93  94  95  96  97
  98  99 140 141 142 143 144 145 146 147 148 149]
교차 검증별 정확도 [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도 0.96


### cross_val_score()

- 교차검증을 보다 간편하게 수행할 수 있게 해주는 함수
- cross_val_score 함수의 파라미터
    - estimator: DecisionTreeClassifier와 같은 모델을 넣어줌
    - X: 피처 데이터 세트
    - y: 타겟 데이터 세트
    - scoring: 예측 성능 평가 기준을 뭘로 할 것인지 accuracy, recall 같은 것
    - cv: fold를 몇 개로 할 것인지 결정하는 것
- 기본적으로 Stratified K-Fold 방식이 내장되어 있다
    

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
target = iris_data.target

# 아래 한 줄의 코드로 위에서 진행한 Stratified K-Fold 검증 과정을 대신할 수 있다
scores = cross_val_score(dt_clf, data, target, scoring="accuracy", cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도: 0.96


### GridSearchCV

- 교차 검증과 하이퍼 파라미터 튜닝을 한 번에 하면서 모델의 성능 향상을 도모하는 클래스
- 교차 검증 점수를 기반으로 하이퍼 파라미터를 모든 경우의 수에 대해 반복 학습하면서 최적의 파라미터 값들의 쌍을 찾는다
- 파라미터1을 3개, 파라미터2를 2개, cv를 3으로 주면 총 3x2x3=18번 학습/검증을 수행하면서 최적의 값을 찾아낸다

In [44]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로딩 & 학습데이터와 테스트 데이터로 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)

# estimator 준비
dtree = DecisionTreeClassifier()

# 하이퍼파라미터를 아래와 같이 dictionary 형태로 준비
parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

In [48]:
# refit=True (default) 로 설정하면 최적의 하이퍼파라미터가 도출되면 마지막에 그걸로 estimator를 학습시킨다
# 그리고 이 best estimator는 best_estimator_ 를 통해 접근할 수 있다
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과가 cv_results_에 딕셔너리로 저장되는데 이를 df로 바꿔준다
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [50]:
print("GridSearchCV 최적 파라미터:", grid_dtree.best_params_)
print("GridSearchCV 최고 정확도:", grid_dtree.best_score_)

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.975


In [55]:
# refit=True로 설정되어서 최적 하이퍼파라미터 쌍을 활용해 학습된 estimator가 내장되어있다
# 따라서 아래와 같이 그대로 predict를 수행할 수 있다
pred = grid_dtree.predict(X_test)
print("실제 테스트 정확도:", np.round(accuracy_score(y_test, pred), 4))

실제 테스트 정확도: 0.9667


In [56]:
# refit으로 최종 학습된 모델은 best_estimator_에 저장되는데 이를 가져와서 predict를 할 수도 있다
estimator = grid_dtree.best_estimator_

# 위 GridSearchCV 객체에서 바로 predict한 것과 동일한 결과를 반환한다
pred = estimator.predict(X_test)
print("실제 테스트 정확도:", np.round(accuracy_score(y_test, pred), 4))

실제 테스트 정확도: 0.9667
